In [1]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
import pyqmc.api as pyq
import pyqmc.tbdm
import pyqmc.obdm
import glob
from os.path import exists
import itertools as it

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def get_descriptors(f, wf_type, wfname, wf_coeffs=None):
    d = {}
    if wf_type == 'eigenstates' and wf_coeffs == None:
        wf = f'{wf_type}/{wfname}'
    else:
        wf = f'{wf_type}/{wfname}/{wf_coeffs}'
    d['energytotal'] = f[f'{wf}/energytotal'][()] - f[f'eigenstates/0/energytotal'][()]
    if wfname != '0':
        d['energytotal_err'] = np.sqrt(f[f'{wf}/energytotal_err'][()]**2+f[f'eigenstates/0/energytotal_err'][()]**2)
    elif wfname == '0':
        d['energytotal_err'] = f[f'{wf}/energytotal_err'][()]

    # occupations
    for i in np.arange(5): 
        d[f'n_{i}_up'] = f[f'{wf}/rdm1_up'][i,i] 
        d[f'n_{i}_up_err'] = f[f'{wf}/rdm1_up_err'][i,i]
        d[f'n_{i}_down'] = f[f'{wf}/rdm1_down'][i,i]
        d[f'n_{i}_down_err'] = f[f'{wf}/rdm1_down_err'][i,i]
    
    # occupations on low-energy and high-energy manifolds
    d[f"n_le"] = d['n_0_up']+d['n_0_down']+d['n_1_up']+d['n_1_down']+d['n_4_up']+d['n_4_down']
    d[f"n_le_err"] = np.sqrt(d['n_0_up_err']**2+d['n_0_down_err']**2+d['n_1_up_err']**2+d['n_1_down_err']**2+d['n_4_up_err']**2+d['n_4_down_err']**2)
    d[f"n_he"] = d['n_2_up']+d['n_2_down']+d['n_3_up']+d['n_3_down']
    d[f"n_he_err"] = np.sqrt(d['n_2_up_err']**2+d['n_2_down_err']**2+d['n_3_up_err']**2+d['n_3_down_err']**2)

    d['trace_up'] = np.einsum('ii', f[f'{wf}/rdm1_up'])
    d['trace_up_err'] = np.sqrt(np.einsum('ii', f[f'{wf}/rdm1_up_err'][...]**2))
    d['trace_down'] = np.einsum('ii', f[f'{wf}/rdm1_down'])
    d['trace_down_err'] = np.sqrt(np.einsum('ii', f[f'{wf}/rdm1_down_err'][...]**2))
    d['trace'] = d['trace_up']+d['trace_down']
    d['trace_err'] = np.sqrt(d['trace_up_err']**2+d['trace_down_err'][...]**2)

    for i in np.arange(5):
        d[f'doccp_{i}'] = f[f'{wf}/rdm2_ud'][i,i,i,i]
        d[f'doccp_{i}_err'] = f[f'{wf}/rdm2_ud_err'][i,i,i,i]
    d["doccp"] = np.sum([d[f'doccp_{i}'] for i in np.arange(5)])
    d["doccp_err"] = np.sqrt(np.sum([d[f'doccp_{i}_err']**2 for i in np.arange(5)]))

    d['sisj'] = 0
    d['sisj_err'] = 0
    for i,j in it.product(np.arange(5), np.arange(5)):
        d['sisj'] += - 2*f[f'{wf}/rdm2_ud'][i,j,j,i]\
        - 2*f[f'{wf}/rdm2_ud'][j,i,i,j]\
        - f[f'{wf}/rdm2_ud'][j,j,i,i]\
        - f[f'{wf}/rdm2_ud'][i,i,j,j]\
        - f[f'{wf}/rdm2_uu'][i,j,j,i]\
        - f[f'{wf}/rdm2_dd'][i,j,j,i] 
        
        d["sisj_err"] += 4*f[f'{wf}/rdm2_ud_err'][i,j,j,i]**2\
        + 4*f[f'{wf}/rdm2_ud_err'][j,i,i,j]**2\
        + f[f'{wf}/rdm2_ud_err'][j,j,i,i]**2\
        + f[f'{wf}/rdm2_ud_err'][i,i,j,j]**2\
        + f[f'{wf}/rdm2_uu_err'][i,j,j,i]**2\
        + f[f'{wf}/rdm2_dd_err'][i,j,j,i]**2

    d['sisj'] = (d['sisj']+9)/4
    d['sisj_err'] = np.sqrt(d['sisj_err'])/4

    d["Sz"] = (d["trace_up"] - d["trace_down"])/2
    d["Sz_err"] = np.sqrt(d["trace_up_err"]**2 + d["trace_down_err"]**2)/2

    return d
    

In [ ]:
vmc_chkfile = "./vcp2_eigenstates_rdm_vmc_data.hdf5"
dmc_chkfile = "./vcp2_eigenstates_rdm_extrapolated_estimators_data.hdf5"

df = []
for chkfile, method in zip([vmc_chkfile, dmc_chkfile], ["vmc", "dmc"]):
    if not exists(chkfile):
        raise FileNotFoundError(f"{chkfile} not found")
    with h5py.File(chkfile, 'r') as f:
        for wf_type in ['eigenstates']:
            for wf_type_component in f[wf_type].keys():
                d = get_descriptors(f, wf_type, wf_type_component)
                d['method'] = method
                d['wf_type'] = wf_type
                d['state'] = int(wf_type_component)

                # spin or charge type excitations 
                # (in the paper charge excitations are called "crystal-field excitations")
                if d["n_he"] > 0.3:
                    d["wf_type2"] = "charge"
                else:
                    d["wf_type2"] = "spin"

                df.append(d)

df = pd.DataFrame(df)

df = df.sort_values(by=['method', 'state']).reset_index(drop=True)
# sort the columns
df = df[['method', 'wf_type', 'wf_type2', 'state', 'energytotal', 'energytotal_err',\
                'n_0_up', 'n_0_up_err', 'n_0_down', 'n_0_down_err',\
                'n_1_up', 'n_1_up_err', 'n_1_down', 'n_1_down_err',\
                'n_2_up', 'n_2_up_err', 'n_2_down', 'n_2_down_err',\
                'n_3_up', 'n_3_up_err', 'n_3_down', 'n_3_down_err',\
                'n_4_up', 'n_4_up_err', 'n_4_down', 'n_4_down_err',\
              'n_le', 'n_le_err', 'n_he', 'n_he_err',\
              'trace', 'trace_err','trace_up', 'trace_up_err', 'trace_down', 'trace_down_err',\
              'doccp_0', 'doccp_0_err', 'doccp_1', 'doccp_1_err', 'doccp_2', 'doccp_2_err',\
              'doccp_3', 'doccp_3_err', 'doccp_4', 'doccp_4_err',\
              'doccp', 'doccp_err', 'sisj', 'sisj_err', 'Sz', 'Sz_err']]
df.to_csv("./vcp2_qmc_descriptors_summary.csv", index=False)

In [4]:
df[["wf_type2", "method", "energytotal", "state", "n_he", "sisj"]]

,wf_type2,method,energytotal,state,n_he,sisj
0,spin,dmc,0.000000,0,0.024088,3.642050
1,spin,dmc,1.229583,1,0.013085,0.782181
2,spin,dmc,1.175560,2,0.018988,0.812191
3,charge,dmc,1.866706,3,0.991585,3.642790
4,charge,dmc,1.855933,4,0.992059,3.615701
5,spin,dmc,1.321323,5,0.019395,0.785503
6,spin,dmc,1.302029,6,0.031803,0.759383
7,charge,dmc,2.025998,7,0.991731,3.634475
8,charge,dmc,2.089256,8,0.992939,3.668549
9,spin,dmc,1.442078,9,0.075641,0.789209
